In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
data = pd.read_excel('Cou_Data.xlsx')
import re
import jieba
jieba.load_userdict("./Cou_dict.txt")


def chinese_word_cut(mytext):
    #jieba.load_userdict('dic.txt')  # 这里你可以添加jieba库识别不了的网络新词，避免将一些新词拆开
    jieba.initialize()
    # 文本预处理 ：去除一些无用的字符只提取出中文出来
    new_data = re.findall('[\u4e00-\u9fa5]+', str(mytext), re.S)
    new_data = " ".join(new_data)

    # 文本分词
    seg_list_exact = jieba.cut(new_data, cut_all=True)
    result_list = []
    with open('Cou_stop.txt','r', encoding='utf-8') as f:  # 可根据需要打开停用词库，然后加上不想显示的词语
        con = f.readlines()
        stop_words = set()
        for i in con:
            i = i.replace("\n", "")  # 去掉读取每一行数据的\n
            stop_words.add(i)

    for word in seg_list_exact:
        if word not in stop_words and len(word) > 1:
            result_list.append(word)
    return " ".join(result_list)


data["report_cutted"] = data.report.apply(chinese_word_cut)
#文本向量化
from sklearn.feature_extraction.text import CountVectorizer

n_features = 2000  # 提取1000个特征词语
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=10)
tf = tf_vectorizer.fit_transform(data.report_cutted)
#构建LDA
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 8
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=200,
                                learning_method='batch',
                                learning_offset=50,
                                doc_topic_prior=0.1,
                                topic_word_prior=0.01,
                               random_state=10000000)  # 关于模型的参数,可查看官方文档
lda.fit(tf)
#输出每个主题对应词语
def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx}:" )
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword
# 输出每个主题对应词语
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
土地 用地 宅基地 房屋 资产 建房 依法 村民 建筑 验收 流转 产权 自然 办事 材料 第二十 办理 行政 合同 备案
Topic #1:
公路 耕地 农村公路 养护 农田 交通 供水 管护 考核 运输 整治 交通运输 自然 水利 地保 高标 高标准 粮食 绿化 基本农田
Topic #2:
培训 教育 卫生 健康 整治 生活 垃圾 村庄 厕所 公共 社区 活动 考核 职业 治理 人居 科学 分类 基层 医疗
Topic #3:
产品 示范 特色 旅游 创新 农产 农产品 科技 文化 平台 品牌 创业 培育 数字 电子 融合 物流 加工 振兴 人才
Topic #4:
应急 储备 粮食 灾害 监测 隐患 执法 排查 处置 风险 交通 发生 行政 预警 事故 防汛 预案 食品 公安 储备粮
Topic #5:
补贴 保险 扶贫 脱贫 贫困 补助 就业 攻坚 农户 振兴 帮扶 涉农 家庭 居民 缴费 医疗 贫困户 劳动 中央 人口
Topic #6:
产品 养殖 农机 农产 农产品 秸秆 种植 畜禽 机械 粮食 作物 作业 示范 品质 补助 产品质量 绿色 补贴 畜牧 有机
Topic #7:
生态 污染 治理 生态环境 污水 土壤 生活 防治 排放 垃圾 环保 用水 水源 整治 监测 城市 环境保护 养殖 污水处理 修复
